In [1]:
#-*- coding:utf-8 -*-

import requests, os
import bs4
from bs4 import BeautifulSoup
from urllib import parse
import re
import lxml
import json
import sys

In [2]:
# return the list of shortcode from html code
def get_shortcode_list(soup):
    soup_script = soup.find_all("script")[3].string
    soup_script = re.split(':|,|\"|\'|\{|\}|\[|\]',soup_script)
    shortcodes = []
    temp = 0
    while 1:
        if 'shortcode' in soup_script[int(temp):]:

            new_shortcode = soup_script.index('shortcode', temp) + 3
            shortcodes.append(soup_script[new_shortcode])
            temp = new_shortcode + 1
        else :
            print("Get ", len(shortcodes), " shortcodes")
            return shortcodes

        
# return the text from html code
def get_text(page):
    script = page.find_all("script")[3].string
    script_split = re.split(':|,|\"|\'|\{|\}|\[|\]',script)  
    text_index = script_split.index('text') + 3
    text = script_split[text_index]  
    
    text = text.replace('\\n','')
    
    text = text.encode('utf-8', 'surrogatepass')
    text = text.decode('unicode_escape')
    
    #print(text)
    
    return text  

# return the user_id and number of likes from html code
def get_id_likes(description):
    data = re.split('on Instagram:', description)

    if '(' in data[0]:
        user_id_index = 6
    else:
        user_id_index = 5

    data = re.split(",|\(|\)|@|Likes|Comments|-",data[0])
    
    likes = data[0]
    user_id = data[user_id_index]
    
    return user_id, likes

# return the text, user_id, number of likes from html code
def get_labels(page):
    for tag in page.select('meta[property*=image]'):
        image_url = tag['content']
            
    for tag in page.select('meta[property*=og:description]'):
        description = tag['content']

    text = get_text(page)
    user_id, likes = get_id_likes(description)
    
    return text, user_id, likes

# save image in page
def save_image(page):
    for tag in page.select('meta[property*=image]'):
        image_url = tag['content']
                
    pic_response = requests.get(image_url).content
    img_name = "./dataset/img/" + word + "_" + shortcode + ".jpg"
    
    with open(img_name, 'wb') as f:
        f.write(pic_response)
    print("file saved: ", img_name)    

In [3]:
word = input("\n** 원하는 태그를 입력하세요: \n>>> ")

# get the url from word(tag)
tag_url = "www.instagram.com/explore/tags/" + word
tag_url = "http://" + parse.quote(tag_url)
print(tag_url)

# get html code
response = requests.get(tag_url).text
soup = BeautifulSoup(response, 'lxml')

# get the list of shortcode
shortcodes = get_shortcode_list(soup)


** 원하는 태그를 입력하세요: 
>>> 남산
http://www.instagram.com/explore/tags/%EB%82%A8%EC%82%B0
Get  74  shortcodes


In [4]:
i = 1
file_name = "./dataset/label.json"

for shortcode in shortcodes:
    print(i, "th ", shortcode)
    i = i + 1  
    label_data = []
    
    try:
        # get html code
        page_url = "https://www.instagram.com/p/" + shortcode + "/"
        response = requests.get(page_url).content
        page = BeautifulSoup(response, 'lxml')        
        
        # get label data
        text, user_id, likes = get_labels(page)
        
        # append new label data 
        output = {"shortcode":shortcode, "label":word, "likes":likes, "user_id":user_id, "text":text}
        label_data.append(output)
        
        if user_id!=" " and text!="" and user_id!="/ ":
            
            if not os.path.isfile(file_name):
                with open(file_name, mode='w', encoding='UTF-16', errors='ignore') as f:
                    new_feed = json.dumps(label_data, ensure_ascii=False, indent=2)
                    f.write(new_feed)
            else:
                with open(file_name, encoding='UTF-16') as feedsjson:
                    old_feeds = json.load(feedsjson)

                old_feeds.append(label_data)
                with open(file_name, mode='w', encoding='UTF-16', errors='ignore') as f:
                    new_feed = json.dumps(old_feeds, ensure_ascii=False, indent=2)
                    f.write(new_feed) 

            # save image files
            save_image(page)
            
            print("Okay\n")
        else:
            print("Not okay\n")
    except:
        print("Unexpected error:", sys.exc_info()[0], sys.exc_info()[1])
        print("Not okay\n")

1 th  Bp3sLEJnc6-
file saved:  ./dataset/img/남산_Bp3sLEJnc6-.jpg
Okay

2 th  Bp3sA7-jgXO
file saved:  ./dataset/img/남산_Bp3sA7-jgXO.jpg
Okay

3 th  Bp3r82Cnn6v
file saved:  ./dataset/img/남산_Bp3r82Cnn6v.jpg
Okay

4 th  Bp3r3DnleL7
file saved:  ./dataset/img/남산_Bp3r3DnleL7.jpg
Okay

5 th  Bp3rzFJhI0w
file saved:  ./dataset/img/남산_Bp3rzFJhI0w.jpg
Okay

6 th  Bp3rbHbloVU
Not okay

7 th  Bp3qz5WHd_P
file saved:  ./dataset/img/남산_Bp3qz5WHd_P.jpg
Okay

8 th  Bp3qeqenkZD
file saved:  ./dataset/img/남산_Bp3qeqenkZD.jpg
Okay

9 th  Bp3qFfIjq0P
file saved:  ./dataset/img/남산_Bp3qFfIjq0P.jpg
Okay

10 th  Bp3qKg9nTMs
file saved:  ./dataset/img/남산_Bp3qKg9nTMs.jpg
Okay

11 th  Bp3p-OJH9Zu
Not okay

12 th  Bp3pwrqBN7R
file saved:  ./dataset/img/남산_Bp3pwrqBN7R.jpg
Okay

13 th  Bp3prL9n1ee
file saved:  ./dataset/img/남산_Bp3prL9n1ee.jpg
Okay

14 th  Bp3pqVcgACa
Unexpected error: <class 'UnicodeDecodeError'> 'unicodeescape' codec can't decode byte 0x5c in position 9: \ at end of string
Not okay

15 th  Bp3pnXNl